# Running your own simulation!

### Step 1: Importing necessary libraries

/Users/ericjankowski/miniforge3/envs/mxene/lib/python3.12/site-packages/gmso/core/element.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


### Step 2: Define flake geometry

### Step 3: Define forcefield, Weeks-Chandler-Anderson

### Step 4: Define simulation parameters

In [12]:
import sys
sys.path.append(".")
sys.path.append("..")
import command_line_sim
from command_line_sim import LJChain, Flake, Pack,get_target_box_number_density, Molecule
import hoomd
import unyt as u
N_chains = 20 # number of polymer chains
initial_dens = 0.001 # initial packing density to initialize system
final_dens = 0.3 # final packing density for shrinking
N_flakes = 5 # number of flakes
chain_length = 10 # length of polymer chains
dt = 0.005 # step size of simulation
temp = 3.0 # kT, temperature of simulation

### Step 5: Running on a CPU? Or a GPU?

In [4]:
device = hoomd.device.CPU() # change to hoomd.device.GPU() if you can/want to run on your GPU.Running

### Step 6: Initialize system

In [13]:
kg_chain = LJChain(lengths=chain_length,num_mols=N_chains) # initializing polymer chains
sheet = Flake(x_repeat=5, y_repeat=5, n_layers=1, periodicity=(False, False, False)) # initializing flakes
system = Pack(molecules=[Molecule(compound=sheet.all_molecules[0], num_mols=N_flakes), kg_chain], 
              density=initial_dens, packing_expand_factor = 6, seed=2) # packing chains and flakes into system
target_box = get_target_box_number_density(density=final_dens*u.Unit("nm**-3"),n_beads=(500+(N_chains*10))) # acquiring final density scaling with number density

### Step 7: Output files

In [7]:
gsd = f"{N_chains}_{chain_length}mer{N_flakes}f_{dt}dt.gsd" # name of output gsd files
log = f"{N_chains}_{chain_length}mer{N_flakes}f_{dt}dt.txt" # name of output log files

### Step 8: Initializing simulation, then running shrink -> simulation

In [8]:
sim = Simulation(initial_state=system.hoomd_snapshot, forcefield=ff.hoomd_forces, device=device, dt = dt, gsd_write_freq=int(1000), log_file_name = log, gsd_file_name = gsd) # initializing simulation
sim.run_update_volume(final_box_lengths=target_box, kT=6.0, n_steps=5e5,tau_kt=100*sim.dt,period=10,thermalize_particles=True) # shrink simulation run
sim.run_NVT(n_steps=1e6, kT=temp, tau_kt=dt*100) # simulation run
sim.flush_writers() # updating data files

*Warning*: dihedral.harmonic: specified K <= 0


Initializing simulation state from a gsd.hoomd.Frame.
Step 1000 of 500000; TPS: 2385.77; ETA: 3.5 minutes
Step 2000 of 500000; TPS: 3750.45; ETA: 2.2 minutes
Step 3000 of 500000; TPS: 4639.58; ETA: 1.8 minutes
Step 4000 of 500000; TPS: 4932.04; ETA: 1.7 minutes
Step 5000 of 500000; TPS: 5408.0; ETA: 1.5 minutes
Step 6000 of 500000; TPS: 5769.96; ETA: 1.4 minutes
Step 7000 of 500000; TPS: 5809.41; ETA: 1.4 minutes
Step 8000 of 500000; TPS: 6067.05; ETA: 1.4 minutes
Step 9000 of 500000; TPS: 6279.46; ETA: 1.3 minutes
Step 10000 of 500000; TPS: 6468.72; ETA: 1.3 minutes
Step 11000 of 500000; TPS: 6435.88; ETA: 1.3 minutes
Step 12000 of 500000; TPS: 6585.46; ETA: 1.2 minutes
Step 13000 of 500000; TPS: 6712.71; ETA: 1.2 minutes
Step 14000 of 500000; TPS: 6823.84; ETA: 1.2 minutes
Step 15000 of 500000; TPS: 6923.6; ETA: 1.2 minutes
Step 16000 of 500000; TPS: 7015.57; ETA: 1.1 minutes
Step 17000 of 500000; TPS: 7098.27; ETA: 1.1 minutes
Step 18000 of 500000; TPS: 7031.78; ETA: 1.1 minutes
Ste

In [ ]:
!mv $gsd $log ../analysis/